# Рассчитываем эмбеддинги для всех видео

## Импорт необходимых библиотек

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch
import re
from html import unescape
import pandas as pd
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import os
from transformers import AutoTokenizer, AutoModel
import torch

## Загрузка и подготовка данных

### Загрузка датасета

In [8]:
data_videoinfo = pd.read_csv('video_info_v2.csv')

### Предобработка названий видео

In [10]:
import re
from html import unescape
def clean_text(text):
    if isinstance(text, str):
        clean = re.compile('<.*?>')
        re.sub(clean, '', text)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'[✔✖️✳️✴️❗❕❓❔❕⚠️‼️⁉️🔘🔲🔳🔴🔵🔶🔷🔸🔹🔺🔻🟥🟧🟨🟩🟦🟪🟫⬛⬜️⬜️⚫⚪⚽🏀🏈⚾🎾🏐🏉🎱🎳🎮🕹️🎲🎯🎳🔧🔨⚙️⚒️🛠️⛏️⛓️⚔️🔫🧰🧲🧪🧫🧩🧠🧴🧷🧹🧻🧼🧽🧯🧰🧲🧪🧫🧩🧠🧴🧷🧹🧻🧼🧽🧯⛔✳️✨⭐🌟💫💥💦💨🕰️🕹️🗝️🗝️🔑🔒🔓🔐🔏🔎🧰🧲🧪🧫🧩🧠🧴🧷🧹🧻🧼🧽🧯⛔✳️✨⭐🌟💫💥💦💨🕰️🕹️🗝️🗝️🔑🔒🔓🔐🔏🔎🧰🧲🧪🧫🧩🧠🧴🧷🧹🧻🧼🧽🧯⛔✳️✨⭐🌟💫💥💦💨🕰️🕹️🗝️🗝️🔑🔒🔓🔐🔏🔎]' , '', text)
        text = re.sub(r'[✔\u200B-\u200D\uFEFF]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'<.*?>', '', text)
        text = unescape(text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = str(text)
        return text.lower()
data_videoinfo['title'] = data_videoinfo['title'].apply(clean_text)
data_videoinfo.head()

,rutube_video_id,title,category,duration,author_id
0,video_185549,как собрать букет из мыльных тюльпанов силикон...,Хобби,1559160,1015054
1,video_111035,осторожно киберземляне 1 сезон 12 серия,Сериалы,1320007,1002180
2,video_476517,популярные видеоигры в lego перевод td bricks,Хобби,606145,1095337
3,video_157198,хороший лжец фильм 2019,Фильмы,6577440,1043618
4,video_289824,нашего старого гнобят повсякому,Развлечения,859493,1009535


In [11]:
data_videoinfo.drop(['category', 'duration', 'author_id'], axis = 1, inplace = True)
data_videoinfo.head()

,rutube_video_id,title
0,video_185549,как собрать букет из мыльных тюльпанов силикон...
1,video_111035,осторожно киберземляне 1 сезон 12 серия
2,video_476517,популярные видеоигры в lego перевод td bricks
3,video_157198,хороший лжец фильм 2019
4,video_289824,нашего старого гнобят повсякому


## Получаем эмбеддинги для наших видео

### Использование предобученной модели tiny-RuBert для получения векторных представлений названий видеороликов

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def get_embeddings(texts, batch_size=64):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=40).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings_list.append(embeddings)
    return torch.cat(embeddings_list)
title_embeddings = get_embeddings(list(data_videoinfo['title'].values))
title_embeddings

tensor([[ 0.3401, -0.8857,  0.2989,  ...,  0.1304, -0.1277, -1.1357],
        [ 0.4569,  0.4863,  0.0974,  ...,  1.2004,  0.9756, -0.6080],
        [-0.5763, -0.5698, -0.0467,  ..., -0.1897,  0.5094, -0.7498],
        ...,
        [ 0.2508, -0.4075, -0.1583,  ...,  0.2961,  0.7759, -1.4605],
        [-0.6975,  0.0696, -0.0055,  ..., -0.0389,  0.2224, -1.0294],
        [ 0.3728, -0.3621, -0.7591,  ...,  0.2010,  0.4537, -1.2406]],
       device='cuda:0')

### Сохраняем итоговую таблицу с эмбеддингами

In [14]:
data_videoinfo['embeddings'] = title_embeddings.tolist()

In [16]:
data_videoinfo.drop(columns = ['title'], inplace = True)

In [17]:
data_videoinfo.head()

,rutube_video_id,embeddings
0,video_185549,"[0.34006547927856445, -0.8857478499412537, 0.2..."
1,video_111035,"[0.45694699883461, 0.48634690046310425, 0.0973..."
2,video_476517,"[-0.576312780380249, -0.5698351860046387, -0.0..."
3,video_157198,"[-0.5488875508308411, -0.25806984305381775, -0..."
4,video_289824,"[0.2059473991394043, -0.5128248333930969, 0.07..."


In [ ]:
data_videoinfo.to_csv('all_emb_tiny.csv')